# 1. 实现多元线性回归(Normal Equation)预测 Boston House Price

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [4]:
X.shape

(490, 13)

490行，13个特征

In [5]:
import sys
sys.path.append("..")

from playML.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, seed = 666)

In [6]:
from playML.LinearRegression import LinearRegression

reg = LinearRegression()
reg.fit_normal(X_train, y_train)

LinearRegression()

In [7]:
# 系数
reg.coef_

array([-1.20354261e-01,  3.64423279e-02, -3.61493155e-02,  5.12978140e-02,
       -1.15775825e+01,  3.42740062e+00, -2.32311760e-02, -1.19487594e+00,
        2.60101728e-01, -1.40219119e-02, -8.35430488e-01,  7.80472852e-03,
       -3.80923751e-01])

In [8]:
# 截距
reg.interception_

34.11739972320428

In [9]:
reg.score(X_test, y_test)

0.8129794056212832

使用Simple Linear Regression 的 R^2 = 0.6129316803937324

<hr/>

# 2. 使用scikit-learn中的LinearRegression

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 666)

In [15]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

**注意，sklearn中不是使用Normal Equation进行预测！**


In [18]:
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [19]:
lin_reg.coef_

array([-1.15625837e-01,  3.13179564e-02, -4.35662825e-02, -9.73281610e-02,
       -1.09500653e+01,  3.49898935e+00, -1.41780625e-02, -1.06249020e+00,
        2.46031503e-01, -1.23291876e-02, -8.79440522e-01,  8.31653623e-03,
       -3.98593455e-01])

In [21]:
# sklearn中train-test-split以及随机化时与自己实现时可能有所不同
lin_reg.intercept_

32.59756158869959

In [23]:
lin_reg.score(X_test, y_test)

0.8009390227581041

# 3.使用scikit-learn中kNN Regressor

In [28]:
from sklearn.neighbors import KNeighborsRegressor

# 使用default parameter k = 5
knn_reg = KNeighborsRegressor()
knn_reg.fit(X_train, y_train)
knn_reg.score(X_test, y_test)

0.602674505080953

**使用gridSearch**

In [32]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {
        "weights": ["uniform"],
        "n_neighbors": [i for i in range(1, 11)]
    },
    {
        "weights": ["distance"],
        "n_neighbors": [i for i in range(1, 11)],
        "p": [i for i in range(1, 6)]
    }
]
knn_reg = KNeighborsRegressor()
grid_search = GridSearchCV(knn_reg, param_grid, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

/Users/jeremy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    0.5s finished
/Users/jeremy/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [33]:
grid_search.best_params_

{'n_neighbors': 6, 'p': 1, 'weights': 'distance'}

**注意：下面grid_search的best_score_与上面的knn_reg.score()求解方式不同,是因为使用了交叉验证(CV)的方式**

In [34]:
grid_search.best_score_

0.6060528490355778

**注意：下面才是与上面的knn_reg.score()求解方式相同的R^2值，
但是注意这些score都是GridSearchCV里的计算方法，与我们独自实现的计算方法不同**

In [39]:
grid_search.best_estimator_.score(X_test, y_test)

0.7353138117643773

**注意：各种不同算法会得到不同的评价标准，比较时要注意可比较性！**